In [1]:
import sys
import os
# Add this line at the very start of your notebook
sys.path.append(os.path.abspath('..'))  # if notebook is in notebooks/ directory
# or
sys.path.append(os.path.abspath('.'))   # if notebook is in root directory

# Then import your modules
from src.Training.trainer import Trainer
from src.Training.solver import Solver
from src.Training.NeuralPINN import PINO
from src.Models.models import PINN_Net, CustomPINN

c:\Users\mohammed\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch

x = torch.linspace(-1, 1, 32 , requires_grad = True)
y = torch.linspace(-1, 1, 32 , requires_grad= True)
X, Y = torch.meshgrid(x, y)
X = X.unsqueeze(-1)
Y = Y.unsqueeze(-1)
u_exact = torch.exp(-10*(X**2 + Y**2))
f_exact = torch.exp(-10*(X**2 + Y**2))*(-20)*(1-20*X**2) + torch.exp(-10*(X**2 + Y**2))*(-20)*(1-20*Y**2)


u_left = u_exact[:, 0]
u_right = u_exact[:, -1]
u_bottom = u_exact[0, :]
u_top = u_exact[-1, :]

u_exact = lambda x, y, p: torch.exp(p*(-x**2 - y**2))
f_exact = lambda x, y, p: (-4*p + 4*p*p*(x*x + y*y)) * torch.exp(p*(-x*x - y*y))

BC = [u_left , u_right , u_bottom , u_top]
exact_u = torch.exp(-1*(X**2 + Y**2))
model = PINO(1,1,100,2)
model.optimizer = torch.optim.Adam(model.model.parameters(), lr=0.01)
model.train(500, X , Y , f_exact , BC , None )

c:\Users\mohammed\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3596.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Epoch 0 Loss 4.558801651000977 , pde loss 4.471793174743652 , BC loss 0.08700846880674362
Epoch 100 Loss 0.00641611497849226 , pde loss 0.0063982680439949036 , BC loss 1.7846734408522025e-05
Epoch 200 Loss 0.0017335939919576049 , pde loss 0.0017227653879672289 , BC loss 1.0828594895428978e-05
Epoch 300 Loss 0.000838779320474714 , pde loss 0.0008292790735140443 , BC loss 9.500270607532002e-06
Epoch 400 Loss 0.0005126578034833074 , pde loss 0.0005034533096477389 , BC loss 9.204472917190287e-06


In [3]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

def derivation( u, x , order = 1):
    for _ in range(order):
        u = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True,retain_graph=True,allow_unused=True)[0]
    return u  
operator = lambda u,x: derivation(u, x, order=2) 
f= lambda x: (-1.6 * (torch.pi**2) * torch.sin(torch.pi * x * 4) - 50 * torch.tanh(5 * x) * (1 - torch.tanh(5 * x)**2))
u_exact = lambda x:  0.1 * torch.sin(torch.pi * x * 4) + torch.tanh(5 * x)
domaine = [-1,1]
coords = [torch.linspace(-1, 1, 100).to(device)]


solver = Solver(
        hidden_size=100,
        num_layers=1,
        learning_rate=0.01,
        device=None,
        wandb_logs=False,
        name=None
)

model , mse , loss = solver.solve(dimensions =1 , 
                                    coords = coords , 
                                    source_function= f , 
                                    operator= operator, 
                                    u_exact= u_exact , 
                                    epochs = 2000 , num_points = 20,
                                    num_test_points = 500 , 
                                    adaptive_weights = True , 
                                    adaptive_collocation = True ,
                                    update_rate = 100 ,
                                    plot = False )

Training:   0%|          | 0/2000 [00:00<?, ?epoch/s]

Training the model...


Training: 100%|██████████| 2000/2000 [00:51<00:00, 38.54epoch/s, Loss=0.0004, MSE=2.3117e-08]
